In [2]:
#data feature
import cv2
import numpy as np
from skimage.feature import local_binary_pattern

# get horitonzal edges from sobelY
def number_of_edgePixels(image):
    sobelY = cv2.Sobel(image, cv2.CV_64F,0,1,ksize=5)
    return np.count_nonzero(sobelY==255)

# std of mean of rows, get high value for black/white horisontal stripes.
def std_meanOfRows(image):
    mean_row = np.mean(image, axis=1)
    return np.std(mean_row, dtype=np.float32)

# mean number of peak and valleys in each column, get high value for noisy pictures.
def mean_PeaksValleys(image):
    peaks_valleys = []
    for j in range(image.shape[1]):
        col = image[:, j]
        num_peaks, num_valleys = count_peaks_valleys(col)
        mean_peaks_valleys = (num_peaks + num_valleys) / 2
        peaks_valleys.append(mean_peaks_valleys)
    return np.mean(peaks_valleys, dtype=np.float32)
    
def count_peaks_valleys(arr_1d):
    num_peaks = 0
    num_valleys = 0
    for i in range(1, len(arr_1d) - 1):
        if arr_1d[i] > arr_1d[i-1] and arr_1d[i] > arr_1d[i+1]:
            num_peaks += 1
        elif arr_1d[i] < arr_1d[i-1] and arr_1d[i] < arr_1d[i+1]:
            num_valleys += 1
    return (num_peaks, num_valleys)

# numbers of edges from canny edge detection, use dfs to explore number of group of edge pixels
def num_edges_canny(image, L2Gradient, sobel_kernal_size):
    T_lower = 100
    T_upper = 200 
    edge = cv2.Canny(image, T_lower, T_upper, apertureSize=sobel_kernal_size, L2Gradient = L2Gradient)
    def dfs(r, c):
        if r < 0 or r >= np.size(image, 0) or c < 0 or c >= np.size(image, 1) or image[r][c] == 0:
            return 0
            
        image[r][c] = 0
            
        for i, j in zip((r - 1, r + 1, r, r), (c, c, c - 1, c + 1)):
            dfs(i, j)
            
        return 1
    
    return sum(dfs(i, j) for i in range(np.size(image, 0)) for j in range(np.size(image, 1)))

def lbp_histogram(image, radius, bins):
    # compute the LBP histogram of the image
    n_points = 8 * radius
    lbp = local_binary_pattern(image, n_points, radius, method='uniform')
    hist, _ = np.histogram(lbp, bins= bins, density=True)
    return np.ravel(hist)

In [16]:
# example of using features to train model
# number_image = 2910
# feature = 4
# img=np.zeros(shape=(number_image,feature))
# for i in range(0,number_image): 
#     img[i][0] = number_of_edgePixels(npdata['data'][i])
#     img[i][1] = std_meanOfRows(npdata['data'][i])
#     img[i][2] = mean_PeaksValleys(npdata['data'][i])
#     img[i][3] = np.std(npdata['data'][i])
# print(img.shape)
# dataset = Bunch(data = img, target=npdata['label'])

(2910, 4)


In [19]:
# svm_model(dataset, 0.2, 'poly', 3)

Classification Training Report: 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       686
           1       0.93      0.88      0.90       625
           2       0.93      0.96      0.94      1017

    accuracy                           0.95      2328
   macro avg       0.95      0.95      0.95      2328
weighted avg       0.95      0.95      0.95      2328

Train Confusion Matrix: 
 [[686   0   0]
 [  2 548  75]
 [  0  42 975]] 

Train Accuracy:  0.9488831615120275


Classification Testing Report: 
               precision    recall  f1-score   support

           0       0.99      1.00      1.00       184
           1       0.95      0.89      0.92       158
           2       0.94      0.97      0.95       240

    accuracy                           0.96       582
   macro avg       0.96      0.95      0.96       582
weighted avg       0.96      0.96      0.96       582

Test Confusion Matrix: 
 [[184   0   0]
 [  1 141  16]
 [  